# ElMo 실습


## 1. 환경구성

In [6]:
pip install tensorflow==1.15 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 412.3 MB 19 kB/s 
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 1.15.2
    Uninstalling tensorflow-1.15.2:
      Successfully uninstalled tensorflow-1.15.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.7 requires tensorflow>=2.0.0, but you have tensorflow 1.15.0 which is incompatible.


In [7]:
pip install tensorflow-hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow.keras import backend as K
import urllib.request
import pandas as pd
import numpy as np

In [6]:
elmo = hub.Module("https://tfhub.dev/google/elmo/1", trainable=True)
# 텐서플로우 허브로부터 ELMo를 다운로드

sess = tf.Session()
K.set_session(sess)
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

## 2. 데이터 로드

In [9]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv", filename="spam.csv")
data = pd.read_csv('spam.csv', encoding='latin-1')
data[:5]

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


여기서 필요한 건 v2열과 v1열입니다. v1열은 숫자 레이블로 바꿔야 할 필요가 있습니다. 이를 각각 X_data와 y_data로 저장하겠습니다.

In [10]:
data['v1'] = data['v1'].replace(['ham','spam'],[0,1])
y_data = list(data['v1'])
X_data = list(data['v2'])

In [11]:
X_data[:5]

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
 'Ok lar... Joking wif u oni...',
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
 'U dun say so early hor... U c already then say...',
 "Nah I don't think he goes to usf, he lives around here though"]

In [12]:
print(y_data[:5])

[0, 0, 1, 0, 0]


In [13]:
print(len(X_data))
n_of_train = int(len(X_data)*0.8)
n_of_test = int(len(X_data)- n_of_train)
print(n_of_train)
print(n_of_test)


5572
4457
1115


전체 데이터는 5,572개이며 8:2로 비율하면 각각 4,457과 1,115가 됩니다. 이를 각각 훈련 데이터와 테스트 데이터의 양으로 하여 데이터를 분할하겠습니다.

In [14]:
X_train = np.asarray(X_data[:n_of_train]) #X_data 데이터 중에서 앞의 4457개의 데이터만 저장
y_train = np.asarray(y_data[:n_of_train]) #y_data 데이터 중에서 앞의 4457개의 데이터만 저장
X_test = np.asarray(X_data[n_of_train:]) #X_data 데이터 중에서 뒤의 1115개의 데이터만 저장
y_test = np.asarray(y_data[n_of_train:]) #y_data 데이터 중에서 뒤의 1115개의 데이터만 저장

## 4. ELMo와 설계 모델을 연결
ElMo는 텐서플로우 허브에서 가져왔기 때문에 케라스에서 사용가능하도록 변환

In [15]:
def ELMoEmbedding(x):
    return elmo(tf.squeeze(tf.cast(x, tf.string)), as_dict=True, signature="default")["default"]
# 데이터의 이동이 케라스 → 텐서플로우 → 케라스가 되도록 하는 함수

In [18]:
# 모델을 설계
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Lambda, Input
input_text = Input(shape=(1,), dtype=tf.string) # 하나의 샘플, 비공식적 샘플 길이
embedding_layer = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text) # 임베딩층(엘모 임베딩 층을 적용, output_shape 지정)
hidden_layer = Dense(256, activation='relu')(embedding_layer)
output_layer = Dense(1, activation='sigmoid')(hidden_layer)
model = Model(inputs=[input_text], outputs=output_layer)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [19]:
history = model.fit(X_train, y_train, epochs=1, batch_size=60)

Train on 4457 samples
4457/4457 [==============================] - 1419s 318ms/sample - loss: 0.1331 - acc: 0.9495


In [20]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

1115/1115 [==============================] - 371s 333ms/sample - loss: 0.0624 - acc: 0.9794

 테스트 정확도: 0.9794
